## Configure your Foundation Model

Setting up your foundation model is beyond the scope of this repository, as there is not a unified method.  
We lean on the protocol used by <ins>[LiteLLM chat completions](https://docs.litellm.ai/docs/completion)</ins> as it provides a consistent method for interacting with a wide variety of providers.  It also makes things "look like" OpenAI so it is expected to need minimal adaptation for a majority of usecases.

Configuration will usually involve specifying how to make authorized calls to your model, so will most frequently be setting secrets in keys and possibly specifying custom urls.

The evaluation framework expects parts from both ends of a completion function.  <br/>
The <ins>[completion function](https://docs.litellm.ai/docs/completion/input)</ins> should be callable and support input arguments of a model specifier, messages array (list of dicts with user+content), and any provider specific configuration.<br/>
Currently two pieces of the <ins>[output json](https://docs.litellm.ai/docs/completion/output)</ins> are expected:  

- `response['choices'][0]['message']['content']` should be the text of the completion
- `response['usage']` should whichever keys in total_tokens, completion_tokens, and prompt_tokens that you might want to limit for an evaluation.

In [ ]:
# Generally you'll need to set up some connections and authorization such tokens or keys.
# In this case all of that is hardcoded in the module

from example_provider import completion

## Get data

Data is messy.  It's rare that there will not be some finer alignment when using a model.  
For the coding-savvy, much of this can be offloaded into a data_prep function.

Common patterns of managing data for these analyses include using pandas dataframes for in memory representation, or serialized to file for more focused access.
This example combines the two to give a starting point that is partially applicable for either. Here our in-memory dataframe is a list of file descriptors, and the prompt creation includes the logic of reading these files before resolving the evaluation message array.

In [ ]:
import pandas as pd
from pathlib import Path

data_path = "/home/seismo/workspace/examples/data/"

input_df = pd.DataFrame({"guid":['00a5d60f-04fb-4b93-bb88-af5a686aa871',
                                 '000df600-cde6-43d3-b94d-07000216cd17',
                                 '000df600-cde6-43d3-b94d-07000216cd17']},
                        index=['000', '00a', '00a-2'])
input_df.index.name = "myIx"
input_df.guid = data_path + input_df.guid

## Setup the instrument

Selection of a pre-formed prompt can be done via module imports. Namely, the data_prep_fn mentioned alongside data as these two parts must be compatible.

Regardless of the data details, the evaluation loop uses pandas.DataFrame.itertuples. <br/>
The data_prep function should take this namedtuple of a single row and produce a single messages array compatible with the second positional argument of the completion function (loaded above).

Then we initialize our evaluator with, at minimum, the completion and preparation function.  <br/>
This can also set some capacity limit on token usage.  While a lot of these options are expected to be handled by the completion provider, the Evaluation class can support aborting the loop after a number of cumulative tokens are exceeded.  This requires that completion return usage total_tokens.


In [ ]:
from pdsqi_prompt import pdsqi_from_file
import evaluation_instruments as ev

ev.set_logging(10) # DEBUG
evaluator = ev.Evaluation(
    completion_fn = completion,
    prep_fn= pdsqi_from_file,
    log_enabled = True,
    max_tokens = 20_000,
)

# Evaluate

Now, all that is left is to run the dataset through it.  
The run_dataset requires a dataframe where one row at a time is evaluated in a very similar manner as a HuggingFace Pipeline, chaining three steps:
- prompt = prep_fn(namedtuple[dataframe itertuples])
- raw_output = completion_fn(model, prompt)
- response, usage = post_process_fn(raw_output)

The default post_process_fn will extract a single completion and assumes a json-style completion.  The function will further try to parse such a json.

The ultimate output of run_dataset is two-fold:
- A dictionary keyed off of the index from the original dataframe to the value from parsing the completion-json (or whatever the first output of post_process_fn returns)
- A total accumulated TokenUsage

If log_enabled is set to True, the run will output all the individual lines of raw_output under evaluation/logs/raw_content_<TIMESTAMP>.jsonl 

In [ ]:
output = evaluator.run_dataset(input_df, model='gpt-4o-mini')

# Inspect

In [ ]:
grades = ev.frame_from_evals(output[0])

In [ ]:
grades.head()